# Show the model training status 
### Imports

In [2]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from lib.utils import *

## Group 1: CelebaA, UTKface age, FairFace age

In [3]:
def get_basic_cm_stat(idx, stat):
        # Inupt: stat in shape (1, attribute, 8)
        mtp, mfp, mfn, mtn = stat[0,idx,0:4]
        ftp, ffp, ffn, ftn = stat[0,idx,4:8]
        # accuracy
        macc = (mtp+mtn)/(mtp+mfp+mfn+mtn)
        facc = (ftp+ftn)/(ftp+ffp+ffn+ftn)
        tacc = (mtp+mtn+ftp+ftn)/(mtp+mfp+mfn+mtn+ftp+ffp+ffn+ftn)
        # fairness
        mtpr, mtnr = mtp/(mtp+mfn), mtn/(mtn+mfp)
        ftpr, ftnr = ftp/(ftp+ffn), ftn/(ftn+ffp)
        equality_of_opportunity = abs(mtpr-ftpr)
        equalized_odds = abs(mtpr-ftpr) + abs(mtnr-ftnr)
        return macc, facc, tacc, equality_of_opportunity, equalized_odds

def get_basic_cm_stat_list(idx, stat):
    # Input: stat in shape (epoch, attribute, 8)
    macc_list, facc_list, tacc_list, eoo_list, eo_list = list(), list(), list(), list(), list()
    for e in range(stat.shape[0]):
        macc, facc, tacc, equality_of_opportunity, equalized_odds = get_basic_cm_stat(idx, stat[e:e+1,:,:])
        macc_list.append(macc)
        facc_list.append(facc)
        tacc_list.append(tacc)
        eoo_list.append(equality_of_opportunity)
        eo_list.append(equalized_odds)
    return macc_list, facc_list, tacc_list, eoo_list, eo_list

def show_celeba_val_stat(val_stat, name='default', root_folder='./eval/celeba'):
    folder = Path(root_folder)
    folder.mkdir(parents=True, exist_ok=True)
    path = folder / f"{name}.png"
    fig, axs  = plt.subplots(2,4, figsize=(18,8))
    attributes = ['Attractive', 'High_Cheekbones', 'Mouth_Slightly_Open', 'Smiling']
    xs = np.linspace(0, val_stat.shape[0]-1, val_stat.shape[0])
    for attr in range(val_stat.shape[1]): # attribute
        macc_list, facc_list, tacc_list, eoo_list, eo_list = get_basic_cm_stat_list(attr, val_stat)
        axs[0][attr].set_title(attributes[attr])
        macc, = axs[0][attr].plot(xs, macc_list)
        facc, = axs[0][attr].plot(xs, facc_list)
        tacc, = axs[0][attr].plot(xs, tacc_list)
        axs[0][attr].set_xlabel('Epochs')
        axs[0][attr].set_ylabel('Accuracy')
        axs[0][attr].legend((macc, facc, tacc), ('Male', 'Female', 'Total'), loc='lower right')
        axs[0][attr].set_ylim([0.5, 1.0])
        #
        eoo, = axs[1][attr].plot(xs, eoo_list)
        eo, = axs[1][attr].plot(xs, eo_list)
        axs[1][attr].set_xlabel('Epochs')
        axs[1][attr].set_ylabel('Fairness, (lower the better)')
        axs[1][attr].legend((eoo, eo), ('equality of opportunity', 'equalized odds'), loc='upper right')
        axs[1][attr].set_ylim([0.0, 1.0])

    fig.tight_layout()
    fig.savefig(path,)
    plt.close(fig)

#
# model_ckpt_root = Path('/tmp2/aislab/makila/model_stats')
# val_stat = load_stats(f'CelebA_val', root_folder=model_ckpt_root/'CelebA')
# show_celeba_val_stat(val_stat, name='CelebA_model')

# load the validation statistics
# the file name might start with: {seed}_{dataset name}_val
# show_celeba_val_stat(val_stat, name='CelebA_model')

In [5]:
def show_agemodel_val_stat(val_stat, name='default', root_folder='./eval/agemodel'):
    folder = Path(root_folder)
    folder.mkdir(parents=True, exist_ok=True)
    path = folder / f"{name}.png"
    fig, axs  = plt.subplots(2,1, figsize=(5,8))
    xs = np.linspace(0, val_stat.shape[0]-1, val_stat.shape[0])
    # only have 1 attribute (age)
    macc_list, facc_list, tacc_list, eoo_list, eo_list = get_basic_cm_stat_list(0, val_stat)
    axs[0].set_title('Is age greater than 30?')
    macc, = axs[0].plot(xs, macc_list)
    facc, = axs[0].plot(xs, facc_list)
    tacc, = axs[0].plot(xs, tacc_list)
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Accuracy')
    axs[0].legend((macc, facc, tacc), ('Male', 'Female', 'Total'), loc='lower right')
    axs[0].set_ylim([0.5, 1.0])
    #
    eoo, = axs[1].plot(xs, eoo_list)
    eo, = axs[1].plot(xs, eo_list)
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Fairness, (lower the better)')
    axs[1].legend((eoo, eo), ('equality of opportunity', 'equalized odds'), loc='upper right')
    axs[1].set_ylim([0.0, 1.0])

    fig.tight_layout()
    fig.savefig(path,)
    plt.close(fig)

#
# model_ckpt_root = Path('/tmp2/aislab/makila/model_stats')
# val_stat = load_stats(f'UTKFaceAge_val', root_folder=model_ckpt_root/'UTKFaceAge')
# show_agemodel_val_stat(val_stat, name='UKFaceAge_model', root_folder='./eval/utkfaceage')
#
# model_ckpt_root = Path('/tmp2/aislab/makila/model_stats')
# val_stat = load_stats(f'FairFaceAge_val', root_folder=model_ckpt_root/'FairFaceAge')
# show_agemodel_val_stat(val_stat, name='FairFaceAge_model', root_folder='./eval/fairfaceage')


# val_stat = load_stats(f'33907_UTKfaceAge_val')
# show_agemodel_val_stat(val_stat, name='UTKfaceAge_model', root_folder='./eval/utkface')
# val_stat = load_stats(f'33907_FairFaceAge_val')
# show_agemodel_val_stat(val_stat, name='FairFaceAge_model', root_folder='./eval/fairface')

## Group 2: UTKface, FairFace

In [6]:
def get_basic_q_stat(idx, stat):
    # Inupt: stat in shape (1, attribute, 4)
    g1acc = stat[0,idx,0] / (stat[0,idx,0]+stat[0,idx,1])
    g2acc = stat[0,idx,2] / (stat[0,idx,2]+stat[0,idx,3])
    tacc  = (stat[0,idx,0]+stat[0,idx,2]) / np.sum(stat[0,idx,:])
    diff = abs(g1acc-g2acc)
    return g1acc, g2acc, tacc, diff

def get_basic_q_stat_list(idx, stat):
    # Input: stat in shape (epoch, attribute, 4)
    g1acc_list, g2acc_list, tacc_list, diff_list = list(), list(), list(), list()
    for e in range(stat.shape[0]):
        g1acc, g2acc, tacc, diff = get_basic_q_stat(idx, stat[e:e+1,:,:])
        g1acc_list.append(g1acc)
        g2acc_list.append(g2acc)
        tacc_list.append(tacc)
        diff_list.append(diff)
    return g1acc_list, g2acc_list, tacc_list, diff_list

def show_faceattrmodel_val_stat(val_stat, name='default', root_folder='./eval/model'):
    folder = Path(root_folder)
    folder.mkdir(parents=True, exist_ok=True)
    path = folder / f"{name}.png"
    fig, axs  = plt.subplots(2,3, figsize=(14,8))
    attributes = ['Race', 'Gender', 'Age']
    xs = np.linspace(0, val_stat.shape[0]-1, val_stat.shape[0])
    for attr in range(val_stat.shape[1]): # attribute
        g1acc_list, g2acc_list, tacc_list, diff_list = get_basic_q_stat_list(attr, val_stat)
        axs[0][attr].set_title(attributes[attr])
        g1acc, = axs[0][attr].plot(xs, g1acc_list)
        g2acc, = axs[0][attr].plot(xs, g2acc_list)
        tacc, = axs[0][attr].plot(xs, tacc_list)
        axs[0][attr].set_xlabel('Epochs')
        axs[0][attr].set_ylabel('Accuracy')
        axs[0][attr].legend((g1acc, g2acc, tacc), ('Group 1', 'Group 2', 'Total'), loc='lower right')
        axs[0][attr].set_ylim([0.5, 1.0])
        #
        diff, = axs[1][attr].plot(xs, diff_list)
        axs[1][attr].set_xlabel('Epochs')
        axs[1][attr].set_ylabel('Fairness, (lower the better)')
        axs[1][attr].legend((diff,), ('Difference in accuracy',), loc='upper right')
        axs[1][attr].set_ylim([0.0, 1.0])

    fig.tight_layout()
    fig.savefig(path,)
    plt.close(fig)

#
model_ckpt_root = Path('/tmp2/aislab/makila/model_stats')
val_stat = load_stats(f'UTKFace_val', root_folder=model_ckpt_root/'UTKFace')
show_faceattrmodel_val_stat(val_stat, name='UTKFace_model', root_folder='./eval/utkface')
#
model_ckpt_root = Path('/tmp2/aislab/makila/model_stats')
val_stat = load_stats(f'FairFace_val', root_folder=model_ckpt_root/'FairFace')
show_faceattrmodel_val_stat(val_stat, name='FairFace_model', root_folder='./eval/fairface')


# val_stat = load_stats(f'33907_UTKface_val')
# show_faceattrmodel_val_stat(val_stat, name='UTKface_model', root_folder='./eval/utkface')
# val_stat = load_stats(f'33907_FairFace_val')
# show_faceattrmodel_val_stat(val_stat, name='FairFace_model', root_folder='./eval/fairface')
